In [183]:
import pandas as pd
import re

In [184]:
game_df = pd.read_csv('../../data/processed/all_game_data.csv')
game_df = game_df[['steam_appid','price_overview','genres','developers','publishers','categories','release_date','metacritic']]
steam_df = pd.read_csv('../../data/processed/all_steam_data.csv')

merge循环设计，抓取steam_df中所有游戏的补充信息

In [185]:
columns_list = steam_df.columns # 定义原始steam表格的列列表

merge_column_list = ['appid_game1','appid_game2','appid_game3','appid_game4','appid_game5','appid_game1_2weeks','appid_game2_2weeks','appid_game3_2weeks'] # 定义需要查询的列表

gather_df = steam_df # 初始化gather_df
suffix_counter = 1 # 初始化后缀计数器

for column in merge_column_list:
    suffix = f'{suffix_counter}' # 动态生成后缀
    gather_df = pd.merge(left=gather_df, right=game_df, left_on=column, right_on='steam_appid', how='left', suffixes=('', suffix))
    suffix_counter += 1 # 更新后缀计数器


更新部分不匹配的后缀

In [186]:
# 初始化参数
fixed_suffix = "_2weeks_game_"
suffix_count = 0
increment_count = 1

# 获取列名列表
columns = gather_df.columns.tolist()

# 修改列名

for i in range(83, len(columns)):
    # 该正则表达式表示将column[i]中（即一个list）出现线序数字数字0-9的替换为空'',并返回一个空字符串（sub）
    base_name = re.sub(r'\d+', '', columns[i]) 
    columns[i] = f'{base_name}{fixed_suffix}{increment_count}'
    suffix_count += 1
    if suffix_count % 8 ==0:
        increment_count += 1

gather_df.columns = columns
    

In [187]:
gather_df.columns.to_list()[43:51]

['steam_appid',
 'price_overview',
 'genres',
 'developers',
 'publishers',
 'categories',
 'release_date',
 'metacritic']

In [188]:
# 整理残余问题
columns  = gather_df.columns.tolist()
suffix = 1
for i in range(43,51):
    columns[i] = f'{columns[i]}{suffix}'

gather_df.columns = columns
gather_df = gather_df.drop(columns=['Unnamed: 0'])
gather_df = gather_df.drop_duplicates(subset='steamid')


In [189]:
gather_df.to_csv('../../data/processed/all_steam_and_game_data_before_clean.csv',index= False)